<a href="https://colab.research.google.com/github/lcalderin12/EPSY-5641-Google-Collab/blob/main/Module_1_Data_Inquiry_Group_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Header
**Group Members:** Jacob Schmitt, Luis Calderin, Rishika Dokula

In [ ]:
# Init
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

# Initial Display
df = pd.read_csv('/content/gdrive/My Drive/Grad School/EPSY 5641/Module 1 Data Inquiry/data_management_training.csv')
df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/Grad School/EPSY 5641/Module 1 Data Inquiry/data_management_training.csv'

# Initial Download Notes
- Looks like each row's unique ID is manager ID (JS)
- Each manager is tied to a department which has it's own satisfaction score and is the same for each common row of managers in that department (JS)
- I wonder if each manager's score is weighted by their number of reports or not (JS)
  - I do not think so. The score of the manager is the average of the scores of 3 of their employees (RD)
- Is the employee_satisfaction_score tied to each manager as an employee or the cumulative of their employees' scores?  Just checked - it is a measure of how satisfied the manager's direct reports are of them in standard deviation units (JS)
- Generally it looks like we are trying to assess the effectiveness of the training based on the subjective responses of the manager satisfaction survey (JS)
- Treatment seems to refer to whether or not a manager has been trained or not (RD)
- These scores are not raw values, they're graded against each other:
*"a manager with an employee satisfaction score of 1.0 is one standard deviation above the mean for all managers."* (JS)
- n_of_reports seems to denote the number of employees that report to one manager (LC).
- Differences in n_of_reports, department_size, and tennure between trained and untrained managers can contribute to selection bias (LC).
- Treatment (D) would be 1 for training and 0 for not (LC).
- Outcome (Y) is employee_satisfaction_score (LC).
- Could department_satisfaction_score influence the employee_satisfaction_score of treated vs untreated??? (LC)
- Plan is to compute mean employee_satisfaction for trained and untrained managers (LC).
- Think about untreated potential outcomes (LC):
  - if high-performing managers voluntarily select training, then untreated potential outcomes of treated group may already be higher? Would this increase bias?
  -I believe it might increase the untreated potential outcomes of the treated group, hence increasing bias because the experiment is not randomly run. (RD)
  - if struggling managers are looking for improvement and therefore are more likely to select training, the untreated potential outcomes may be lower?


In [ ]:
# Code Sandbox

### All employees - training vs non-training
#grouped_df = df[["training", "employee_satisfaction_score"]].groupby("training").mean()

###grouped by both training and department
#grouped_df = df[["department_id", "training", "employee_satisfaction_score"]].groupby(["department_id","training"]).mean()

###grouped by both training and tenure
#grouped_df = df[["tenure", "training", "employee_satisfaction_score"]].groupby(["tenure","training"]).count()

###compare treatment vs comparison groups
grouped_df = df[["tenure", "department_size", "n_of_reports", "training", "employee_satisfaction_score"]].groupby("training").mean()

grouped_df

,tenure,department_size,n_of_reports,employee_satisfaction_score
training,,,,
0,5.026151,862.185565,4.301674,-0.234678
1,5.474247,928.774728,4.303867,0.199921


# Post-sandbox Notes
- Seems like the obvious place to start would be looking at the score of all training vs all non-training to show what I assume will look like a clear positive correlation with training outcomes but then we should group by department ID or another variable to try to filter out possible areas for selection bias to occur (JS)
- When grouping by both training and department id the eye test still seems to pass - training employee scores still appear higher than non-training scores (JS)
- When grouping by both training and tenure the eye test still seems to pass - trained managers still appear to score higher than their non-trained peers with the same tenure (JS)
- When I adjusted the grouping to look at count as opposed to mean, I noticed that a little more than half (54%) of the managers opted to take the training. The voluntary nature of this training make me think that selection bias is at play. Better managers who score better on subjective questionaires are also probably more likely to sign up for the voluntary training. (JS)



In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

# Initial Display
df_lc = pd.read_csv('/content/gdrive/MyDrive/EPSY 5641/Module 1/data_management_training.csv')
df_lc.head()

Mounted at /content/gdrive


,manager_id,department_id,department_size,department_satisfaction_score,tenure,n_of_reports,training,employee_satisfaction_score
0,0,76,843,0.224077,6,4,1,0.277359
1,1,76,843,0.224077,4,8,1,-0.449646
2,2,76,843,0.224077,6,4,1,0.769703
3,3,76,843,0.224077,6,4,1,-0.121763
4,4,76,843,0.224077,6,4,1,1.526147


In [3]:
#compare significant covariates

#startt with years of tenure
tenure = df_lc.groupby("tenure")["employee_satisfaction_score"].mean()
tenure


,employee_satisfaction_score
tenure,
1,-0.666991
2,-0.663952
3,-0.716852
4,-0.612303
5,-0.269938
6,0.330792
7,0.981158


## Tenure Notes
- Satisfaction is strongly increasing with tenure.(LC)
- Years 1-5 are negative.(LC)
- year 6 is positve.(LC)
- Year 7 is strongly positive.(LC)
- Tenure is positively associated with employee satisfaction.(LC)
- If tenure increases satisfaction even without training, the untreated potential outcome would likely be high.(LC)

In [4]:
#check satisfaction scores with number of reports
reports = df_lc.groupby("n_of_reports")["employee_satisfaction_score"].mean()
reports

,employee_satisfaction_score
n_of_reports,
1,0.052971
2,0.051210
3,-0.281650
4,-0.059870
5,0.561978
6,0.077338
7,-0.002576
8,-0.017822


## Number of Reports Notes
- Satisfaction scores bounce around.(LC)
- Expected from JS code of mean by treatment.(LC)
- Suggests no strong sytemic relationship.(LC)

In [5]:
#department_size
dept_size = df_lc.groupby("department_size")["employee_satisfaction_score"].mean()
dept_size. head()

,employee_satisfaction_score
department_size,
32,-0.223996
43,0.658499
75,0.334183
110,-0.701117
121,0.125714


In [6]:
dept_size.tail()

,employee_satisfaction_score
department_size,
1280,-0.417852
1394,0.124175
1493,-0.146592
1732,-0.258624
2366,0.063462


In [7]:
#looking for avg dept size for negative scores
df_lc[df_lc.employee_satisfaction_score < 0]["department_size"].mean()

np.float64(896.7333958724203)

In [8]:
#looking for avg dept size for positive scores
df_lc[df_lc.employee_satisfaction_score > 0]["department_size"].mean()

np.float64(899.6271487848252)

##Dept Size Notes
- Does not seem to show any correlation and looks iconcistent. (LC)
- Compared average size of department by negative and positve employtee satisfaction score.(LC)
  - satisfaction < 0 = 896.73
  - satisfaction > 0 = 899.63
- The difference is miniscule and shows no meaningful relationship.(LC)
- Dept size is unlikely going to affect untreated potential outcome.(LC)
- Diffrence probably does not contirubute much selection bias.(LC)

# Memo instructions:
Write a one to two page memo - including tables and/or figures but excluding references and appendices - responding to the claims of your HR department. The audience of the memo should be your colleagues who have familiarity with causal inference. They know the terms (e.g., selection bias) but still need the argument fully and clearly explained.

# Guiding questions:
- What is the naïve/observed difference in outcomes between groups?
***The training improved manager satisfaction scores: 0.20 is better than -0.23 (JS)***
- Compare the characteristics of the treatment and comparison groups. ***Managers who have taken the training tend to come from larger departments, have slightly more tenure and receive higher employee satisfaction scores. There does not appear to be a difference in the number of reports. (JS)***
- Do you think these characteristics are more likely related to higher or lower untreated potential outcomes? Explain your reasoning. (If you cannot decide, generate two potential hypotheses).
  -These characteristics are likely related to higher untreated potential outcomes. Greater tenure and training can result in better management and experience of employees working under their respective manager (hence increased satisfaction at the workplace). Larger departments may have greater funding for training, which makes sense that the managers who recieved training come from these departments rather than smaller ones. (RD)
  -Clarification: I am under the impression that a high untreated potential outcome means that the manager already has the advantages to peform better. If it was low, the manager would not have the training and larger tenure to help them recieve better scoring? (RD)
- To what extent do you believe, given all of your answers above, that the department training has a positive effect on the outcome?
  -There was an slight improvement with the outcome jumping from 0.20 to -0.23 as a result of the training. While this was a positive, I am not sure if it is statistically large enough to be considered as highly effective. (RD)
- When you get to next steps, you should certainly consider causal validity but may also consider other validity concerns.
  -Causal validity: yes because training did increase employee satisfaction (RD)
  -Measurement validity:
  -Generalizability validity: training did seem to improve the scoring, but the statistical significance may be too low to generalize (RD)
  -Statiscal validity: yes because there seems to be a casual relationship between the variables, and is likely not due to chance (RD)